In [ ]:
# Include all packages
import gc
import cv2
import shutil
import numpy as np
import pandas as pd

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/DataSet1.zip', 'r') as zip_ref:
        zip_ref.extractall('./DataSet1')
except:
    print("Using Local Machine")

In [ ]:
def CannyEdge(capturedImage):
    grayScale = cv2.cvtColor(capturedImage, cv2.COLOR_BGR2GRAY)
    bilateralFilter = cv2.bilateralFilter(grayScale, 11, 17, 17)
    imageMedian = np.median(capturedImage)
    lowerThreshold = max(0, (0.7 * imageMedian))
    upperThreshold = min(255, (0.7 * imageMedian))
    cannyEdgeImage = cv2.Canny(bilateralFilter, lowerThreshold, upperThreshold)
    cannyEdgeImage = cv2.cvtColor(cannyEdgeImage, cv2.COLOR_GRAY2BGR)
    return cannyEdgeImage

In [ ]:
def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    widthScale = newWidth / originalWidth
    heightScale = newHeight / originalHeight
    resizedImage = cv2.resize(
        image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    x1New, y1New = int(x1 * widthScale), int(y1 * heightScale)
    x2New, y2New = int(x2 * widthScale), int(y2 * heightScale)
    return resizedImage, x1New, y1New, x2New, y2New


In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/annotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=",")
    uniqueSigns = annotationsDataFrame['class'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        images.append(image)
        annotations.append(
            [row[1], row[2], row[3], row[4]])

    del annotationsDataFrame

    return images, annotations, len(uniqueSigns)


In [ ]:
def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = []
    newAnnotations = []
    for i, image in enumerate(images):
        [x1, y1, x2, y2] = annotations[i]
        resizedImage, x1New, y1New, x2New, y2New = ResizeImage(
            image, x1, y1, x2, y2, resize[0], resize[1])
        resizedImage = CannyEdge(resizedImage)
        resizedImages.append(resizedImage)
        newAnnotations.append(
            [x1New, y1New, x2New, y2New])

    
    return resizedImages, newAnnotations


In [ ]:
batchSize = 32
inputShape = (640, 640)

In [ ]:
images, annotations, numClasses = LoadDataSet("./DataSet1")
numClasses = 1

In [ ]:
images, annotations, = PreProcessDataSet(
    images, annotations, batchSize, inputShape)

gc.collect()

In [ ]:
annotationsDataFrame = pd.DataFrame(annotations, columns=['x1','y1', 'x2','y2'])

In [ ]:
annotationsDataFrame.head()
